In [12]:
import tensorflow as tf 
import numpy as np 

path="parches/270324_05/"


inp_tr     = tf.convert_to_tensor(np.load(path+"inp_tr_27_03_24.npy"))[:600,1,:,:,:]
target_tr  = tf.convert_to_tensor(np.load(path+"target_tr_27_03_24.npy"))[:600,0,:,:,:]
lat_tr     = tf.convert_to_tensor(np.load(path+"lat_tr_27_03_24.npy"))[:600,1,:,:,:]
lon_tr     = tf.convert_to_tensor(np.load(path+"lon_tr_27_03_24.npy"))[:600,1,:,:,:]
orog_tr    = tf.convert_to_tensor(np.load(path+"orog_tr_27_03_24.npy"))[:600,1,:,:,:]

inp_va     = tf.convert_to_tensor(np.load(path+"inp_va_27_03_24.npy"))[:600,1,:,:,:]
target_va  = tf.convert_to_tensor(np.load(path+"target_va_27_03_24.npy"))[:600,0,:,:,:]
lat_va     = tf.convert_to_tensor(np.load(path+"lat_va_27_03_24.npy"))[:600,1,:,:,:]
lon_va     = tf.convert_to_tensor(np.load(path+"lon_va_27_03_24.npy"))[:600,1,:,:,:]
orog_va    = tf.convert_to_tensor(np.load(path+"orog_va_27_03_24.npy"))[:600,1,:,:,:]


print("Dimensiones input: tr-", inp_tr.shape, "va-",inp_va.shape)
print("Dimensiones target: tr-", target_tr.shape, "va-", target_va.shape)
print("Dimensiones lat: tr-", lat_tr.shape, "va-", lat_va.shape)
print("Dimensiones lon: tr-", lon_tr.shape, "va-", lon_va.shape)
print("Dimensiones orog: tr-", orog_tr.shape, "va-", orog_va.shape)

Dimensiones input: tr- (600, 32, 32, 1) va- (600, 32, 32, 1)
Dimensiones target: tr- (600, 32, 32, 1) va- (600, 32, 32, 1)
Dimensiones lat: tr- (600, 32, 32, 1) va- (600, 32, 32, 1)
Dimensiones lon: tr- (600, 32, 32, 1) va- (600, 32, 32, 1)
Dimensiones orog: tr- (600, 32, 32, 1) va- (600, 32, 32, 1)


#### Arquitectura U-net

In [20]:

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p  

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


def red_unet(**kwargs):
    
    inputs=[]
    
    for name, shape in kwargs.items():
        input_layer = Input(shape=shape, name=name)
        inputs.append(input_layer)


    # Preprocess input1 with a convolution
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs[0])

    #pop. 
    real_inputs=inputs[:]
    real_inputs.pop(0)
    real_inputs.insert(0, conv1)

    print("real inputs: ",real_inputs)
    print("inputs: ", inputs)

    # Concatenación de inpust
    merged = concatenate([inp for inp in real_inputs], axis=-1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(merged)
    #c1 = BatchNormalization()(c1)  
    #c1 = Activation("relu")(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    ###          ------          ###

    ###          Bloq 2          ###
    s2, p2 = encoder_block(p1, 128)
    ###          Bloq 3          ###
    s3, p3 = encoder_block(p2, 256)
    ###          Bloq 4          ###
    s4, p4 = encoder_block(p3, 512)

    ###       Transición         ###
    b1 = conv_block(p4, 1024)

    ###          Bloq 4          ###
    d1 = decoder_block(b1, s4, 512)
    ###          Bloq 3          ###
    d2 = decoder_block(d1, s3, 256)
    ###          Bloq 2          ###
    d3 = decoder_block(d2, s2, 128)
    ###          Bloq 1          ###
    d4 = decoder_block(d3, c1, 64)

    ###          Salida          ###
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(d4)  ### Serifi: {activation: linear, padding: valid, kernel_size: (3,1,1)}


    model = Model(inputs=inputs, outputs=outputs)

    return model


model_unet = red_unet(input1=(32, 32, 1), input2=(32, 32, 1), input3=(32, 32, 1))
model_unet.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse', 'mae', 'mape'])
model_unet.summary()

real inputs:  [<KerasTensor shape=(None, 32, 32, 64), dtype=float32, sparse=False, name=keras_tensor_427>, <KerasTensor shape=(None, 32, 32, 1), dtype=float32, sparse=None, name=input2>, <KerasTensor shape=(None, 32, 32, 1), dtype=float32, sparse=None, name=input3>]
inputs:  [<KerasTensor shape=(None, 32, 32, 1), dtype=float32, sparse=None, name=input1>, <KerasTensor shape=(None, 32, 32, 1), dtype=float32, sparse=None, name=input2>, <KerasTensor shape=(None, 32, 32, 1), dtype=float32, sparse=None, name=input3>]


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input1 (InputLayer) │ (None, 32, 32, 1) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_176 (Conv2D) │ (None, 32, 32,    │        640 │ input1[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input2 (InputLayer) │ (None, 32, 32, 1) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input3 (InputLayer) │ (None, 32, 32, 1) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_45      │ (None, 32, 32,    │          0 │ conv2d_176[0][0], │
│ (Concatenate)       │ 66)               │            │ input2[0][0],     │
│                     │                   │            │ input3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_177 (Conv2D) │ (None, 32, 32,    │     38,080 │ concatenate_45[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_36    │ (None, 16, 16,    │          0 │ conv2d_177[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_178 (Conv2D) │ (None, 16, 16,    │     73,856 │ max_pooling2d_36… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        512 │ conv2d_178[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_65       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_179 (Conv2D) │ (None, 16, 16,    │    147,584 │ activation_65[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        512 │ conv2d_179[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_66       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_37    │ (None, 8, 8, 128) │          0 │ activation_66[0]… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_180 (Conv2D) │ (None, 8, 8, 256) │    295,168 │ max_pooling2d_37… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 8, 256) │      1,024 │ conv2d_180[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_67       │ (None, 8, 8, 256) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_181 (Conv2D) │ (None, 8, 8, 256) │    590,080 │ activation_67[0]

 Total params: 31,054,785 (118.46 MB)

 Trainable params: 31,043,265 (118.42 MB)

 Non-trainable params: 11,520 (45.00 KB)

In [21]:
history = model_unet.fit(
    {'input1': inp_tr, 'input2': lat_tr, 'input3': lon_tr},  
    target_tr,
    epochs=2,
    batch_size=32,
    validation_data=({'input1': inp_va, 'input2': lat_va, 'input3': lon_va}, target_va)
)


Epoch 1/2
19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 773ms/step - loss: 0.0620 - mae: 0.1967 - mape: 374.6975 - mse: 0.0620 - val_loss: 0.0347 - val_mae: 0.1538 - val_mape: 100.3575 - val_mse: 0.0347
Epoch 2/2
19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 752ms/step - loss: 0.0067 - mae: 0.0650 - mape: 130.7928 - mse: 0.0067 - val_loss: 0.0345 - val_mae: 0.1537 - val_mape: 99.9234 - val_mse: 0.0345
